# Detecting and Mitigating Memory Corruption Errors

## Memory corruption
- https://cwe.mitre.org/data/definitions/787.html
- according to MITRE, "memory corruption" is often used to describe the consequences of writing to memory outside the bounds of a buffer that is invalid, when the root cause is something other than a sequential copy of excessive data from a fixed starting location. This may include issues such as incorrect pointer arithmetic, accessing invalid pointers due to incomplete initialization or memory release, etc.

- from programmers point-of-view, there are two main ways to detect memory corruption errors in C/C++ programs
- White box and black box testing

## White box testing
- also called static analysis
- have access to source code
- manually read and review source code for memory related errors such as memory leak, buffer overflow, underflow, etc.
    - pros and cons?
- can use automated tools to scan for code and API that leads to memory related errors
    - pros and cons?

## Black box testing
- also called dynamic analysis
- manually test the binary/executable
- employ `fuzz testing` - use automated tools called fuzzer to provide invalid, unexpected or random data as inputs to the program

### Use 3rd party scanners such as Valgrind's Memcheck
- https://valgrind.org/docs/manual/quick-start.html
- compile your program using -g (dubuggin info) and -o1 (line numbers in error message)
    - `-o0` is also a good idea, if you can tolerate the slowdown   
- must install valgrind and libc6-dbg:i386 packages

### Use gcc/g++ compiler flags

### NOTE: Automated tools are not perfect!

In [ ]:
! echo kali | sudo -S apt install valgrind -y

In [2]:
# check valgrind version
! valgrind --version

valgrind-3.15.0


In [ ]:
%%bash
# install libc6-dbg:i386 for debugging x86 program in x64
echo kali | sudo -S sudo apt install libc6-dbg -y
echo kali | sudo -S sudo apt install libc6-dbg:i386 -y

In [3]:
! pwd

/workspaces/SoftwareSecurity/notebooks


In [4]:
%cd ../demos/mem_leak

/workspaces/SoftwareSecurity/demos/mem_leak


In [5]:
# let's use demos/memory_leak.cpp program for demo
! cat memory_leak.cpp

  #include <stdlib.h>
  #include <cstring>
  #include <cstdio>

  void f(char * arg)
  {
	 // C dynamic memory
	 int* x = (int *)malloc(10 * sizeof(int));
	 // C++ dynamic memory
	 char* name = new char[20];
	 
	 x[10] = 0;        // problem 1: heap block overrun
		                // problem 2: memory leak -- x not freed
	 strcpy(name, arg);
	 // problem 3: heap block overrun
	 // problem 4: memory leak -- x not freed
	 printf("Hello %s\n", arg);
  }

  int main(int argc, char* argv[1])
  {
	 // what if f() is called over and again in an infinite loop, e.g. 
	 f(argv[1]);
	 return 0;
  }


In [6]:
# compile with -g -o0 options to use with valgrind
# compile as 64-bit binary as valgrind will not work on 32-bit due to lack of 
# libc6-dbg:i386 library
! g++ -g -o0 memory_leak.cpp -o memory_leak.exe

In [7]:
# Run the program with an argument
! ./memory_leak.exe John

Hello John


In [9]:
# program crashes or behaves unexpectedly
! ./memory_leak.exe "some very very very very long string"

malloc(): corrupted top size


In [10]:
# by default gives summary of memory leak
# doesn't give the detail/full memory leaks info
! valgrind ./memory_leak.exe "John Smith"

==6012== Memcheck, a memory error detector
==6012== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==6012== Using Valgrind-3.15.0 and LibVEX; rerun with -h for copyright info
==6012== Command: ./memory_leak.exe John\ Smith
==6012== 
==6012== Invalid write of size 4
==6012==    at 0x1091DD: f(char*) (memory_leak.cpp:12)
==6012==    by 0x109236: main (memory_leak.cpp:23)
==6012==  Address 0x4daaca8 is 0 bytes after a block of size 40 alloc'd
==6012==    at 0x483B7F3: malloc (in /usr/lib/x86_64-linux-gnu/valgrind/vgpreload_memcheck-amd64-linux.so)
==6012==    by 0x1091C2: f(char*) (memory_leak.cpp:8)
==6012==    by 0x109236: main (memory_leak.cpp:23)
==6012== 
Hello John Smith
==6012== 
==6012== HEAP SUMMARY:
==6012==     in use at exit: 60 bytes in 2 blocks
==6012==   total heap usage: 4 allocs, 2 frees, 73,788 bytes allocated
==6012== 
==6012== LEAK SUMMARY:
==6012==    definitely lost: 60 bytes in 2 blocks
==6012==    indirectly lost: 0 bytes in 0 blocks
==6012==      

In [11]:
! valgrind --leak-check=full -s ./memory_leak.exe "John Smith"

==6043== Memcheck, a memory error detector
==6043== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==6043== Using Valgrind-3.15.0 and LibVEX; rerun with -h for copyright info
==6043== Command: ./memory_leak.exe John\ Smith
==6043== 
==6043== Invalid write of size 4
==6043==    at 0x1091DD: f(char*) (memory_leak.cpp:12)
==6043==    by 0x109236: main (memory_leak.cpp:23)
==6043==  Address 0x4daaca8 is 0 bytes after a block of size 40 alloc'd
==6043==    at 0x483B7F3: malloc (in /usr/lib/x86_64-linux-gnu/valgrind/vgpreload_memcheck-amd64-linux.so)
==6043==    by 0x1091C2: f(char*) (memory_leak.cpp:8)
==6043==    by 0x109236: main (memory_leak.cpp:23)
==6043== 
Hello John Smith
==6043== 
==6043== HEAP SUMMARY:
==6043==     in use at exit: 60 bytes in 2 blocks
==6043==   total heap usage: 4 allocs, 2 frees, 73,788 bytes allocated
==6043== 
==6043== 20 bytes in 1 blocks are definitely lost in loss record 1 of 2
==6043==    at 0x483C583: operator new[](unsigned long) (in /usr

## gcc/g++ Warning flags and AddressSanitizer
- https://en.wikipedia.org/wiki/AddressSanitizer
- https://gcc.gnu.org/onlinedocs/gcc-3.4.6/gcc/Warning-Options.html#Warning-Options

- `-Wall` - display all the warning
- `-Wpedantic` - display nonstandard warnings
- `-Wextra` - print extra newer warning messages
- `-Wconversion` - warning any implicit type conversions
- Warnings are like static analysis

- `-fsanitize=address` - use address sanitizer (ONLY works on Linux)
- must compile and run the program to see the results of any buffer-over-flow errors (dynamic analysis)
- For more: https://www.osc.edu/resources/getting_started/howto/howto_use_address_sanitizer

In [ ]:
! g++ -std=c++17 -g -o0 -Wall -Wpedantic -Wextra -Wconversion -fsanitize=address memory_leak.cpp -o memory_leak.exe

In [ ]:
# run the program to see the Address Sanitizer's result
# detects overflow during run-time
! ./memory_leak.exe

In [2]:
# let's compile demos/stack_overflow/so_stdio.cpp with address sanitize flag and warning
! g++ -std=c++17 -m32 -g -o0 -Wall -Wpedantic -Wextra -Wconversion -fsanitize=address ../../stack_overflow/so_stdio.cpp -o so_stdio.exe

g++: error: ../../stack_overflow/so_stdio.cpp: No such file or directory
g++: fatal error: no input files
compilation terminated.


In [ ]:
# let's manually test it... perhaps string not long enough
! echo "here you go some long long long string..." | ./so_stdio.exe

In [ ]:
# just overflow BUFFSIZE of 128
! python -c 'print("A"*200)' | ./so_stdio.exe

## Fixing memory leak and over-run vulnerabilities
- find the vulnerable line of code/functions, etc. and fix it
- see `demos/mem_leak/memory_leak_fixed.cpp` for demo

In [ ]:
! cat demos/memory_leak_fixed.cpp

In [ ]:
# compile with -g -o0 options to use with valgrind
! g++ -g -o0 -Wpedantic -Wextra -Wconversion -fsanitize=address demos/memory_leak_fixed.cpp -o memory_leak_fixed.exe

In [ ]:
# manually check the fix
! ./memory_leak_fixed.exe "some very very very very long strin adfa asf afaf adfa dag"

In [ ]:
# check with valgrind
! valgrind --leak-check=yes ./memory_leak_fixed.exe "some very very very very long string"